In [40]:
import mediapipe as mp
import cv2
import numpy as np
import csv
from matplotlib import pyplot as plt
import argparse
import os
import warnings
warnings.filterwarnings("ignore")

In [41]:

def export_landmark(results, action, path_to_csv):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark],dtype=object).flatten()
        keypoints = np.insert(keypoints,0, action)
        with open(path_to_csv, 'a',newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(e)
        pass


In [42]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose 
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
path_to_csv = 'lunges2.csv'
if not os.path.exists(path_to_csv):
    with open(path_to_csv, 'w') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(landmarks)

In [43]:
video_path = 'lunges_video.mp4'
cap = cv2.VideoCapture(video_path)
marks = []
with mp_pose.Pose(min_detection_confidence=0.75, min_tracking_confidence=0.75) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )
            cv2.rectangle(image, (0,0), (280,220), (245,117,16), -1)
            cv2.putText(image, 'Press u: Up',
                            (10,60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, 'Press l: Left',
                            (10,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, 'Press r: Right',
                            (10,160), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, 'Press q: Quit',
                            (10,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)
            k = cv2.waitKey(1)
            if k == ord('u'):
                export_landmark(results, 'up', path_to_csv)
            elif k == ord('l'):
                export_landmark(results, 'left', path_to_csv)
            elif k == ord('r'):
                export_landmark(results, 'right', path_to_csv)
            # else:
            #     export_landmark(results, 'none', path_to_csv)

            cv2.imshow('Annotation Feed', image)
            if k == ord('q'):
                break
        else:
            print("End of video",ret)
            break

cap.release()
cv2.destroyAllWindows()




End of video False
